# Data Collection

In [ ]:
# Block - 01

from hand_detector import HandDetector
import os
import cv2
import numpy as np
import pandas as pd

In [ ]:
# Block - 02

urdu_signs_mapper = {
    "Alif": 1,       # ا
    "Bay": 2,        # ب
    "Pay": 3,        # پ
    "Tay": 4,        # ت
    "Ttay": 5,       # ٹ
    "Say": 6,        # ث
    "Chay": 7,       # چ
    "Hay": 8,        # ح
    "Khay": 9,       # خ
    "Daal": 10,      # د
    "Ddaal": 11,     # ڈ
    "Zaal": 12,      # ذ
    "Raa": 13,       # ر
    "Zaa": 14,       # ز
    "Saa": 15,       # ژ
    "Seen": 16,      # س
    "Sheen": 17,     # ش
    "Saad": 18,      # ص
    "Zaad": 19,      # ض
    "Tua": 20,       # ط
    "Zua": 21,       # ظ
    "Ain": 22,       # ع
    "Ghien": 23,     # غ
    "Faa": 24,       # ف
    "Qaaf": 25,      # ق
    "Kaaf": 26,      # ک
    "Ghaaf": 27,     # گ
    "Laam": 28,      # ل
    "Meem": 29,      # م
    "Noon": 30,      # ن
    "Wao": 31,       # و
    "Hamza": 32,     # ء
    "Choti_yeh": 33, # ے
    "Bari_yeh": 34,  # ی
    "Hah": 35,       # ہ
    "No Sign": 36    # No Sign
}

urdu_signs_mapper = {value:key for key, value in urdu_signs_mapper.items()}
urdu_signs_mapper

In [ ]:
# Block - 03

class DataCollection:
    
    def __init__(self):
        self.urdu_sign_mapper = None
        self.hand_sign_coordinates = dict()
        self.detector = HandDetector(maxHands=2)
        self.cap = None
    
    def capture_images(self, label:int) -> list:
        count = 0
        temp = []
        self.cap = cv2.VideoCapture(0)
        while True:
            success, frame = self.cap.read()
            key = cv2.waitKey(1)
            if key == ord("q"):
                return temp
            if success:
                hands, frame = self.detector.findHands(frame, draw=True)
                if hands:
                    hand = hands[0]
                    if key == ord("s"):
                        landmarks = self.preprocess(np.array(hand['lmList']).flatten())
                        temp.append(landmarks)
                        count += 1
            else:
                break
            cv2.putText(frame, f"Images: {count}", (45, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            # cv2.putText(frame, f"Label: {urdu_sign_mapper[label]}", (45, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(frame, f"Label: {label}", (45, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow("image", frame)
        return temp
    
    def preprocess(self, landmarks):
        base_x, base_y, base_z = landmarks[0], landmarks[1], landmarks[2]
        for idx, value in enumerate(landmarks):
            if idx in range(0, 62, 3):
                landmarks[idx] -= base_x
            elif idx in range(1, 62, 3):
                landmarks[idx] -= base_y
            else:
                landmarks[idx] -= base_z
        
        max_val = max(list(map(abs, landmarks)))
        landmarks = landmarks / max_val
        return landmarks
    
    def destroy(self):
        self.cap.release()
        cv2.destroyAllWindows()       

In [ ]:
# Block - 04

filename = "temp.csv"
if not os.path.exists(filename):
    df = pd.DataFrame().to_csv(filename)
else:
    df = pd.read_csv(filename)
urdu_signs_mapper = {int(key) : value for key, value in urdu_signs_mapper.items()}
signs = {key:[] for key in urdu_signs_mapper.keys()}
signs = dict()
temp_df = pd.DataFrame()

**Capturing Images Workflow**

1. **Input `1`**  
   Start the image collection process.

2. **Enter the Label**  
   - Provide the label for the letter/sign (refer to the `urdu_signs_mapper` dictionary in *Block - 02*).  
   - Example: If capturing images for "Bay," enter `2` (or its corresponding Urdu letter).

3. **Start Collecting Images**  
   - Press `s` to begin capturing and storing images for the selected label.

4. **Stop Capturing Images**  
   - Press `q` to stop the image collection for the current label.

5. **Save Data**  
   - Input `0` to save the captured images and associated label data to your system.

6. **Exit or Continue**  
   - Input `0` again to exit the process.  
   - Input any other number to restart the collection for another letter/sign.

In [ ]:
# Block - 05

dc = DataCollection()
option = int(input("Press 1 to start."))
collected_labels = []
temp_df = pd.DataFrame()
while option == 1:
    label = int(input("Enter label: "))
    if (label <= 0) or (label > 36):
        print("You entered wrong label....")
        continue
    temp = dc.capture_images(label)
    choice = int(input("Press 0 to save your result"))
    if choice == 0:
        signs[label] = []
        for array in temp:
            signs[label].append(array)
            collected_labels.append(label)
        temp = []
    else:
        temp = []
    option = int(input("Press 0 to exit."))
dc.destroy()

In [ ]:
# Block - 06

for label in pd.Series(collected_labels).unique().tolist():
    temp = pd.DataFrame(signs[label])
    temp['target'] = label
    temp_df = pd.concat([temp_df, temp], axis=0, ignore_index=True)

In [ ]:
# Block - 07

temp_df.head()

If you have gathered some data before and want to concatenate it with currently collected data, run `Block - 08`

In [ ]:
# Block - 08

df = pd.read_csv("handSign.csv")
columns = {str(i):i for i in range(63)}
columns['target'] = 'target'
df.rename(columns=columns, inplace=True)
df = pd.concat([df, temp_df], axis=0)
df.to_csv("handSign.csv", index=False)

In [ ]:
# Block - 09

temp_df.to_csv("handSign.csv", index=False)